<a href="https://colab.research.google.com/github/moonsejin315/Python/blob/main/LangChain/LangChain_03_%EB%8C%93%EA%B8%80_%EB%B6%84%EB%A5%98_%EB%B0%8F_%EC%83%9D%EC%84%B1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai
!pip install langchain
!pip install langchain_community
!pip install langchain_openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.0/337.0 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 632.6 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 377.3/377.3 kB 20.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 9.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.1/141.1 kB 9.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.2/49.2 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.1/47.1 kB 864.9 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/

In [30]:
import os
import openai

from google.colab import userdata
os.environ["OPENAI_API_KEY"] = userdata.get("OPEN_AI_KEY")

openai.api_key = os.getenv("OPENAI_API_KEY")

In [3]:
#대화형 프롬프트의 양식을 지정
from langchain.prompts.chat import ChatPromptTemplate

#모델의 결과값을 원하는 방식으로 파싱(형태 변형)
from langchain.schema import BaseOutputParser

In [4]:
chat_prompt_template = ChatPromptTemplate.from_messages([
    #openai처럼 role-content의 순서로 튜플을 전달하여 요청
    ("system", '''댓글에 대해 감정 분석을 수행할 것. 주어진
                    user-input comment에 대해 긍정적인 경우 1, 부정적인 경우 0. 오로지 0과 1로만 답할 것'''),
    ("human",  "{input_text}" ),
])

In [8]:
#BaseoutputParser를 상속하여 새로운 클래스인 ClassificatioinOutputParser를 정의
#이 클래스는 결과에 대해, str형태의 1과
class ClassificationOutputParser(BaseOutputParser):
  def parse(self, text:str):
    if '1' in text:
      return 1
    return 0

In [31]:
#LangChain에서 OpenAI의 대화형 모델을 사용하여 대화를 생성
from langchain.chat_models import ChatOpenAI

#langchain에서 openai랑 대화할 모델의 정의
chat_model = ChatOpenAI()

#chain을 통해서 양식 생성 -> 모델에 전달 -> 결과값 정리 까지를 한꺼번에 처리
chain = chat_prompt_template | chat_model | ClassificationOutputParser()

In [14]:
def langchain_llm(input_text):
  output = chain.invoke({"input_text" : input_text})
  return output

In [15]:
def classify_text(input_text):
  output = langchain_llm(input_text)
  return output

In [19]:
import urllib.request


urllib.request.urlretrieve(
    "https://raw.githubusercontent.com/tykimos/tykimos.github.io/master/warehouse/dataset/tarr_train.txt",
    filename="tarr_train.txt",
)


('tarr_train.txt', <http.client.HTTPMessage at 0x7e27c4d67b50>)

In [23]:
import pandas as pd

df = pd.read_csv("tarr_train.txt", sep="\t")
df.head()

,id,comment,label
0,1,여기 음식은 언제 와도 실망시키지 않아요. 최고!,1
1,2,여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.,1
2,3,"진짜 깔끔하고, 맛도 좋았어요. 추천합니다!",1
3,4,왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음,0
4,5,인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!,1


In [32]:
actual_labels = []
predicted_labels = []

total = len(df)

for index, row in df.iterrows():
  comment = row['comment']
  actual_label = row['label']

  #llm의 질의 응답을 받아옴
  predicted_label = classify_text(comment)

  actual_labels.append(actual_label)
  predicted_labels.append(predicted_label)

  print(f"[{index+1}/{total}]")
  print("comment " , comment)
  print("actual " , actual_label)
  print("predict" , predicted_label)

  if index > 10:
    break

[1/300]
comment  여기 음식은 언제 와도 실망시키지 않아요. 최고!
actual  1
predict 1
[2/300]
comment  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
actual  1
predict 1
[3/300]
comment  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
actual  1
predict 1
[4/300]
comment  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
actual  0
predict 0
[5/300]
comment  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
actual  1
predict 1
[6/300]
comment  메뉴 설명을 너무 친절하게 해주셔서 고르기 수월했어요.
actual  1
predict 1
[7/300]
comment  사진과 음식이 너무 달라서 실망했습니다.
actual  0
predict 0
[8/300]
comment  주변에 추천하려고 사진도 많이 찍었어요. 좋아요!
actual  1
predict 1
[9/300]
comment  솔직히...? 맛이 그닥이에요. 리뷰랑 너무 다르네.
actual  0
predict 0
[10/300]
comment  진짜 개꿀맛..ㅠ 다른곳 안가.
actual  1
predict 1
[11/300]
comment  음식이 너무 늦게 나와서 기다리는 동안 답답했습니다.
actual  0
predict 0
[12/300]
comment  음식도 맛있고, 가격도 합리적이에요. 다음에 또 오려구요.
actual  1
predict 1


## 댓글에 대한 답변 생성

In [38]:
chat_prompt_reply = ChatPromptTemplate.from_messages([
    ("system", "너는 친절한 사장님이야. 긍정적인 댓글에는 감사함을 구체적으로 표현하고, 부정적인 댓글에는 미안함을 표현하면서 개선 방향도 함께 말해줘"),
    ("human", "{input_text}")
])

In [39]:
from langchain.schema.output_parser import StrOutputParser

chain = chat_prompt_reply | chat_model | StrOutputParser()

def generate_reply(input_text):
  output = langchain_llm(input_text)
  return output

In [40]:
for index, row in df.iterrows():
    comment = row['comment']
    reply = generate_reply(comment)

    print(f"[{index+1}]/[{total}]")
    print("comment : ", comment)
    print("reply : ", reply)
    print("---------------")

    if index > 3:
        break

[1]/[300]
comment :  여기 음식은 언제 와도 실망시키지 않아요. 최고!
reply :  감사합니다! 소중한 의겢을 전해주셔서 정말 감사합니다. 계속해서 최고의 서비스를 제공할 수 있도록 최선을 다하겠습니다. 앞으로도 맛있는 음식으로 만족시켜드릴 수 있도록 노력하겠습니다. 감사합니다!
---------------
[2]/[300]
comment :  여기 라멘 진짜 ㄹㅇ 맛있어요. 국물이 진하고 면도 쫄깃해서 너무 좋았습니다.
reply :  감사합니다! 맛있게 드셔주셔서 정말 기쁘네요. 더 맛있는 음식을 제공할 수 있도록 최선을 다하겠습니다. 앞으로도 많은 사랑 부탁드려요. 😊🍜
---------------
[3]/[300]
comment :  진짜 깔끔하고, 맛도 좋았어요. 추천합니다!
reply :  감사합니다! 소중한 의겢 공유 감사드립니다. 더 나은 서비스를 제공할 수 있도록 노력하겠습니다. 다음 방문 때에도 만족하실 수 있도록 최선을 다하겠습니다. 함께해 주셔서 감사합니다!
---------------
[4]/[300]
comment :  왜 이렇게 유명한지 모르겠음ㅋㅋ ㄹㅈㄷ 맛없음
reply :  미안합니다. 맛에 대한 기대를 충족시키지 못했던 것 같아 안타깝습니다. 더 좋은 서비스를 제공할 수 있도록 향후에는 더욱 노력하겠습니다. 혹시 다른 음식이나 서비스에 대한 의견이 있으시다면 말씀해주시면 감사하겠습니다.
---------------
[5]/[300]
comment :  인생 타르트를 여기서 만났어요❤️ 달지 않고 고소해서 정말 추천합니다!
reply :  감사합니다! 인생 타르트를 맛있게 즐겨주셔서 정말 기쁘네요. 계속해서 맛있는 디저트를 제공할 수 있도록 노력하겠습니다. 앞으로도 많은 사랑 부탁드려요! ❤️
---------------
